In [1]:
from fractions import Fraction

In [2]:
f = Fraction(2, 3)

In [3]:
f.numerator

2

In [4]:
f.denominator

3

In [5]:
f.speak()

AttributeError: 'Fraction' object has no attribute 'speak'

In [6]:
# adding an attribute dynamically at run time
# this is called "monkey patching"
Fraction.speak = lambda self, message: 'Fraction says {0}'.format(message)

In [8]:
f.speak('hello')

'Fraction says hello'

In [9]:
f2 = Fraction(10, 5)

In [10]:
f2.speak('yo!')

'Fraction says yo!'

In [11]:
Fraction.is_integral = lambda self: self.denominator == 1

In [16]:
f1 = Fraction(2, 3)

In [17]:
f2 = Fraction(8, 1)

In [18]:
f1.is_integral()

False

In [19]:
f2.is_integral()

True

In [20]:
# decorator for classes
def dec_speak(cls):
    # mutating the class object
    cls.speak = lambda self, message: '{0} says: {1}'.format(self.__class__.__name__, message)
    return cls

In [21]:
Fraction = dec_speak(Fraction) # equivalent to @dec_speak notation

In [22]:
f1 = Fraction(2, 3)

In [23]:
f1.speak('hello')

'Fraction says: hello'

In [24]:
class Person:
    pass

In [25]:
Person = dec_speak(Person) # equivalent to @dec_speak notation

In [26]:
p = Person()

In [27]:
p.speak('this works!')

'Person says: this works!'

In [28]:
from datetime import datetime, timezone

In [38]:
def info(self):
    results = []
    results.append('time: {0}'.format(datetime.now(timezone.utc)))
    results.append('class: {0}'.format(self.__class__.__name__))
    results.append('id: {0}'.format(hex(id(self))))
    for k, v in vars(self).items():
        results.append('{0}: {1}'.format(k, v))
    return results

def debug_info(cls):
    cls.debug = info
    return cls # note: even though we are modifying the class directly, we have to return the class to use dec syntax

In [39]:
@debug_info
class Person:
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = birth_year
        
    def say_hi():
        return 'Hello there!'

In [40]:
p = Person('John', 1939)

In [41]:
p.debug()

['time: 2019-01-26 06:31:32.818635+00:00',
 'class: Person',
 'id: 0x108e54080',
 'name: John',
 'birth_year: 1939']

In [45]:
@debug_info
class Automobile:
    def __init__(self, make, model, year, top_speed):
        self.make = make
        self.model = model
        self.year = year
        self.top_speed = top_speed
        self._speed = 0
        
    @property
    def speed(self):
        return self._speed
    
    @speed.setter
    def speed(self, new_speed):
        if new_speed > self.top_speed:
            raise ValueError('Speed cannot exceed top_speed.')
        else:
            self._speed = new_speed

In [46]:
favorite = Automobile('Ford', 'Model T', 1908, 45)

In [47]:
favorite.debug()

['time: 2019-01-26 06:35:45.195611+00:00',
 'class: Automobile',
 'id: 0x108e54630',
 'make: Ford',
 'model: Model T',
 'year: 1908',
 'top_speed: 45',
 '_speed: 0']

In [48]:
favorite.speed = 40

In [49]:
favorite.debug()

['time: 2019-01-26 06:35:58.346388+00:00',
 'class: Automobile',
 'id: 0x108e54630',
 'make: Ford',
 'model: Model T',
 'year: 1908',
 'top_speed: 45',
 '_speed: 40']

In [50]:
from math import sqrt

In [51]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)
    
    def __repr__(self):
        return 'Point({0}, {1})'.format(self.x, self.y)

In [52]:
p1, p2, p3 = Point(2, 3), Point(2, 3), Point(0, 0)

In [53]:
p1

Point(2, 3)

In [54]:
p1 is p2

False

In [55]:
p2 is p3

False

In [57]:
p1 == p2 # no __eq__ method defined, so by default Python uses memory address

False

In [62]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)
    
    def __repr__(self):
        return 'Point({0}, {1})'.format(self.x, self.y)
    
    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return False
        
    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented

In [63]:
p1, p2, p3 = Point(2, 3), Point(2, 3), Point(0, 0)

In [64]:
p1 is p2

False

In [65]:
p1 == p2

True

In [66]:
p3 > p1

False

In [67]:
p4 = Point(100, 100)

In [68]:
p4 < p1

False

In [69]:
p4 > p1 # no __gt__ method defined, so Python runs p1 < p4

True

In [70]:
p1 <= p4 # not implemented, but given we've implemented eq and lt, we can use a decorator to build the rest of these

TypeError: '<=' not supported between instances of 'Point' and 'Point'

In [71]:
"""
a <= b iff a < b or a == b
a > b iff not(a < b) and a != b
a >= b iff not (a < b)
"""

'\na <= b iff a < b or a == b\na > b iff not(a < b) and a != b\na >= b iff not (a < b)\n'

In [72]:
def complete_ordering(cls):
    if '__eq__' in dir(cls) and '__lt__' in dir(cls):
        cls.__le__ = lambda self, other: self < other or self == other
        cls.__gt__ = lambda self, other: not (self < other) and not (self == other)
        cls.__ge__ = lambda self, other: not (self < other)
    return cls

In [73]:
@complete_ordering
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)
    
    def __repr__(self):
        return 'Point({0}, {1})'.format(self.x, self.y)
    
    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return False
        
    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented

In [74]:
p1, p2, p3, p4 = Point(2, 3), Point(2, 3), Point(0, 0), Point(100, 200)

In [75]:
p1 <= p4

True

In [76]:
p4 >= p2

True

In [77]:
p1 != p2

False

In [78]:
# this functionality is in functools in stdlib
from functools import total_ordering

@total_ordering
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)
    
    def __repr__(self):
        return 'Point({0}, {1})'.format(self.x, self.y)
    
    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return False
        
    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented

In [79]:
p1, p2, p3, p4 = Point(2, 3), Point(2, 3), Point(0, 0), Point(100, 200)

In [80]:
p1 <= p4

True